## CS310 Natural Language Processing
## Lab 5 (part 1): Data preparation for implementing an RNN Language Model


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### Process input sequences of variable lengths

When training RNN (LSTM or vanilla-RNN), it is difficult to batch the variable length sequences. 

For example: if the length of sequences in a size 8 batch is `[4,6,8,5,4,3,7,8]`, you will pad all the sequences and that will result in 8 sequences of length 8. You would end up doing 64 computations (8x8), but you needed to do only 45 computations. 

PyTorch allows us to pack the sequence, internally packed sequence is a tuple of two lists. One contains the elements of sequences. Elements are interleaved by time steps (see example below) and other contains the size of each sequence the batch size at each step. 

This is helpful in recovering the actual sequences as well as telling RNN what is the batch size at each time step. 

**Example**:

In [3]:
seqs = [torch.tensor([1,2,3]), torch.tensor([3,4])] # Sequences
seq_lens = torch.tensor([3,2]) # Actual lengths of sequences

# First, pad the sequences to the same length
padded_seqs = nn.utils.rnn.pad_sequence(seqs, batch_first=True)

# Then pack them all before passing to the RNN
packed_seqs = nn.utils.rnn.pack_padded_sequence(padded_seqs, seq_lens, batch_first=True, enforce_sorted=False)

# Print intermediate results
print('original sequences:', seqs)
print('padded sequences:', padded_seqs)
print('padded sequences dimension:', padded_seqs.size())
print('packed sequences:', packed_seqs)

original sequences: [tensor([1, 2, 3]), tensor([3, 4])]
padded sequences: tensor([[1, 2, 3],
        [3, 4, 0]])
padded sequences dimension: torch.Size([2, 3])
packed sequences: PackedSequence(data=tensor([1, 3, 2, 4, 3]), batch_sizes=tensor([2, 2, 1]), sorted_indices=tensor([0, 1]), unsorted_indices=tensor([0, 1]))


Note that 
- Default padding ID is 0
- The padded sequence is of shape `batch_size x max_length`. Assuming it is word ids, then after it is embedded, it will be of shape `batch_size x max_length x embedding_size`. 
- Here, `max_length` is the length of the longest sequence in the batch. 
- We set `enforce_sorted` to `False` in `pack_padded_sequence` because we are not sorting the sequences by length. 

---

In the next cell, we will first embed the padded sequence (integer word ids) and then pack the embedded sequence. 

It is the packed embedded sequence that we pass to RNN. It will internally unpack the sequences and compute only the necessary time steps. 

To examine the output, you need to unpack it, which is a reverse process of packing.

In [4]:
embedding = nn.Embedding(5, 10)
rnn = nn.RNN(10, 20, batch_first=True) # 10: input size, 20: hidden size(output size)

with torch.no_grad():
    # padded_seqs: 2*3
    padded_embs = embedding(padded_seqs) # 2*3*10
    packed_embs = nn.utils.rnn.pack_padded_sequence(padded_embs, seq_lens, batch_first=True, enforce_sorted=False)

    out_packed, _ = rnn(packed_embs) 
    out_unpacked, _ = nn.utils.rnn.pad_packed_sequence(out_packed, batch_first=True)


print('padded emb dim:', padded_embs.size())
print('packed output dim:', out_packed.data.size())
print('unpacked output', out_unpacked.size())

padded emb dim: torch.Size([2, 3, 10])
packed output dim: torch.Size([5, 20])
unpacked output torch.Size([2, 3, 20])


Note that 
- `pad_packed_sequence` does the reverse of `pack_padded_sequence`.
- the unpacked output is of shape `batch_size x max_length x hidden_size`, in which the first dimensions match the shape of the padded input sequences.

---

### T1. Practice Padding and Packing

First, Read all text data and build the vocabulary. 

Note that this time the ids for actual words will start from 1, as 0 will be used for padding, i.e., the special token '[PAD]'.

In [5]:
input_file = 'lunyu_20chapters.txt'

# You can use the code from previous lab or rewrite it
# Hint: you can comment out the `self.initTableNegatives()` in `__init__` method
from utils import CorpusReader
corpus = CorpusReader(inputFileName=input_file, min_count=1)


Total vocabulary: 1352


In [6]:

### START YOUR CODE ###
# Modify word2id to make 0 as the padding token '[PAD]', and increase the index of all other words by 1
# Modify the id2word list to make the first word '[PAD]' as well
# Hint: Both word2id and id2word in utils.CorpusReader are dict objects
word2id = {}
word2id['[PAD]'] = 0
for word, id in corpus.word2id.items():
    word2id[word] = id + 1

id2word = {}
id2word[0] = '[PAD]'
for id, word in corpus.id2word.items():
    id2word[id + 1] = word
### END YOUR CODE ###

In [7]:

# Test result
print('id2word:', sorted(list(id2word.items()), key=lambda x: x[0])[:5])
print('word2id:', sorted(list(word2id.items()), key=lambda x: x[1])[:5])

# You should expect to see:
# id2word: [(0, '[PAD]'), (1, '，'), (2, '子'), (3, '。'), (4, '：')]
# word2id: [('[PAD]', 0), ('，', 1), ('子', 2), ('。', 3), ('：', 4)]

id2word: [(0, '[PAD]'), (1, '，'), (2, '子'), (3, '。'), (4, '：')]
word2id: [('[PAD]', 0), ('，', 1), ('子', 2), ('。', 3), ('：', 4)]


Read the first 16 lines of text, and convert them into integer sequences (`torch.Long`) of variable lengths. 

Then, follow the steps of `pad -> embed -> pack` to obtain the packed embedded sequence. 

Pass it to the RNN and then unpack the output.

*Hint*:
- You need to define the `embedding_lunyu` as an `nn.Embedding` object, with the correct vocabulary size and **embedding size of 50**.
- Create the `rnn_lunyu` as an `nn.RNN` object, with the correct input size and **hidden size of 100**.

In [8]:
### START YOUR CODE ###
with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()[:16]

In [9]:
line_words = [list(line.strip()) for line in lines]
seq_ids = [torch.tensor([word2id.get(word, 0) for word in words]) for words in line_words]
seq_lens = torch.tensor([len(ids) for ids in seq_ids])
seq_ids_padded = nn.utils.rnn.pad_sequence(seq_ids, batch_first=True)
seq_ids_padded.size()

torch.Size([16, 85])

In [10]:
embedding_lunyu = nn.Embedding(len(word2id), 50) # vocab_size, embedding_dim
rnn_lunyu = nn.RNN(50, 100, batch_first=True)

In [11]:
seq_embs = embedding_lunyu(seq_ids_padded)
seq_embs_packed = nn.utils.rnn.pack_padded_sequence(seq_embs, seq_lens, batch_first=True, enforce_sorted=False)

In [12]:
out_packed, _ = rnn_lunyu(seq_embs_packed)
out_unpacked, _ = nn.utils.rnn.pad_packed_sequence(out_packed, batch_first=True)
### END YOUR CODE ###


In [13]:

# Test result
print('seq_ids_padded:', seq_ids_padded.size())
print('seq_embs:', seq_embs.size())
print('out_unpacked:', out_unpacked.size())

# You should expect to see:
# seq_ids_padded: torch.Size([16, 85])
# seq_embs: torch.Size([16, 85, 50])
# out_unpacked: torch.Size([16, 85, 100])

seq_ids_padded: torch.Size([16, 85])
seq_embs: torch.Size([16, 85, 50])
out_unpacked: torch.Size([16, 85, 100])


Lastly, map the output of the RNN to the vocabulary size.

*Hint*:
- Define a linear layer `fc` with the correct input (hidden size of RNN) and output size (vocabulary size).
- The output of `fc` will be of shape `batch_size x max_length x vocab_size`, which we call `logits`.
- `logits` are not normalized, so you need to apply `F.log_softmax` to get the log probabilities.

In [14]:
### START YOUR CODE ###
fc = nn.Linear(100, len(word2id))
logits = fc(out_unpacked)
log_probs = F.log_softmax(logits, dim=-1)
### END YOUR CODE ###

# Test result
print('logits:', logits.size())
print('log_probs:', log_probs.size())

# You should expect to see:
# logits: torch.Size([16, 85, 1353])

logits: torch.Size([16, 85, 1353])
log_probs: torch.Size([16, 85, 1353])


### T2. Prepare Target Labels

Prepare the target labels for the RNN. The target labels are the same as the input sequences, but shifted by one time step.

For example, if the input sequences is `[[1, 2, 3], [3, 4, 0]]`, the target labels should be `[[2, 3, 0], [4, 0, 0]]`, where 0 is the padding ID.

In this practice, you need to prepare the target labels for first 16 lines, i.e., `seq_ids_padded`

In [15]:
seq_ids_padded[0]

tensor([  2,   5,   4,  47,   9, 225, 545,   6,   1,   7,  66, 131,  20,  10,
         15, 267, 132, 106, 179, 246,   1,   7,  66,  64,  20,  10,  12,   7,
         30,   9,   7, 546,   1,   7,  66,  19,   2,  20,  10,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0])

In [16]:
### START YOUR CODE ###
targets_padded = torch.zeros_like(seq_ids_padded)
padding_id = 0

for i in range(seq_ids_padded.size(0)):
    targets_padded[i, :-1] = seq_ids_padded[i, 1:] # Shift the sequence to the left by 1
    targets_padded[i, -1] = padding_id # Set the last token to be the padding token

### END YOUR CODE ###

# Test result
print('targets_padded:', targets_padded.size())
print('last column of targets_padded:', targets_padded[:, -1])

print('seq_ids_padded[0]:', seq_ids_padded[0])
print('targets_padded[0]:', targets_padded[0])
# You should expect to see:
# targets_padded: torch.Size([16, 85])
# last column of targets_padded: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

targets_padded: torch.Size([16, 85])
last column of targets_padded: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
seq_ids_padded[0]: tensor([  2,   5,   4,  47,   9, 225, 545,   6,   1,   7,  66, 131,  20,  10,
         15, 267, 132, 106, 179, 246,   1,   7,  66,  64,  20,  10,  12,   7,
         30,   9,   7, 546,   1,   7,  66,  19,   2,  20,  10,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0])
targets_padded[0]: tensor([  5,   4,  47,   9, 225, 545,   6,   1,   7,  66, 131,  20,  10,  15,
        267, 132, 106, 179, 246,   1,   7,  66,  64,  20,  10,  12,   7,  30,
          9,   7, 546,   1,   7,  66,  19,   2,  20,  10,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0, 

### T3. Compute Perplexity

In order to compute the perplexity, we first need to compute the negative log probabilities.

This can be accomplished by using the `nn.NLLLoss` function, which takes the `log_probs` and the `target_labels` as input, and the negative log probability (cross-entropy) loss, averaged over all the non-padding tokens: $-\sum \log(p)$

However, the default output of `nn.NLLLoss` is reduced to the average over all the tokens, including the padding tokens. We need to exclude the padding token by setting the `ignore_index` argument to the padding ID, i.e., 0. Also, set the `reduction` argument to `'none'` to get the loss for each non-padding token.

Finally, compute the perplexity by exponentiating the average loss per sequence.

See the documentation here: https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html


In [17]:
loss_fn = nn.NLLLoss(ignore_index=0, reduction='none')

### START YOUR CODE ###
# Calculate the loss

# log_probs.view(-1, len(word2id)) 将 log_probs 改变形状，使其变成一个二维张量，
learning_rate = 0.1

with torch.no_grad():
    loss = loss_fn(log_probs.view(-1, len(word2id)), targets_padded.view(-1))
    perplexity = torch.exp(loss.mean())
### END YOUR CODE ###

# Test result
print('loss:', loss.size())
print('perplexity:', perplexity)

# You shoul expect to see:
# loss: torch.Size([1360])
# Here, 1360 = 16 * 85, i.e., the total number of tokens in the batch

loss: torch.Size([1360])
perplexity: tensor(24.8225)


### Model Architecture

In `__init__` method, initialize `word_embeddings` with a pretrained embedding weight matrix loaded. For example, the one obtained from previous assignment (saved word2vec file). 

`nn.Embedding` has a method `from_pretrained` that takes the pretrained weight matrix (a `numpy.ndarray` object) to initialize its weight.

`forward` method takes the word id sequences and sequence lengths as inputs, and return the logits or log probabilities from RNN. 

In [18]:
from gensim.models import KeyedVectors

In [19]:
word_vectors = KeyedVectors.load_word2vec_format('embedding_emb50_neg5_win3.txt', binary=False)
embedding_w2v = nn.Embedding.from_pretrained(torch.FloatTensor(word_vectors.vectors))

embedding_rand = nn.Embedding(len(word2id), 50)

embedding_w2v.num_embeddings, embedding_rand.num_embeddings

(1352, 1353)

In [20]:
class RNNLM(nn.Module):
    def __init__(self, embedding: nn.Embedding ,**kwargs):
        super(RNNLM, self).__init__()
        self.embedding = embedding
        self.rnn = nn.RNN(embedding.embedding_dim, hidden_size=100, batch_first=True)
        self.fc = nn.Linear(100, len(word2id))
        

    def forward(self, seq, seq_lens): # pass in raw word ids and sequence lengths
        padded_seqs = nn.utils.rnn.pad_sequence(seq, batch_first=True)
        padded_embs = self.embedding(padded_seqs)
        packed_embs = nn.utils.rnn.pack_padded_sequence(padded_embs, seq_lens, batch_first=True, enforce_sorted=False)
        out_packed, _ = self.rnn(packed_embs)
        out_unpacked, _ = nn.utils.rnn.pad_packed_sequence(out_packed, batch_first=True)
        logits = self.fc(out_unpacked)
        log_probs = F.log_softmax(logits, dim=-1)
        return log_probs

### Sentence Generation

After training the RNN, we can use it to generate sentences. 

The process is as follows:
- Start with a special token or a sequence of tokens, e.g., ["子", "曰"]
- Pass the sequence to the RNN, and sample the next word from the output probability distribution of the last time step. We use greedy search here, i.e., select the word with the highest probability.
- Append the sampled word to the sequence, and repeat the process until a special token, e.g., "。", is sampled; Or until it reaches the maximum length of generation.

In [21]:
seq = ['子','曰']
model = RNNLM(embedding_w2v)
max_length = 10

In [22]:

model.eval()
with torch.no_grad():
    current_tokens = seq
    for _ in range(max_length):
        current_tokens_tensor = torch.tensor([[word2id[word] for word in current_tokens]])
        seq_lens = torch.tensor([len(current_tokens)])
        # 调用模型，获取下一个单词的概率分布
        log_probs = model(current_tokens_tensor, seq_lens)
        # 从概率分布中采样下一个单词的索引
        next_word_index = torch.argmax(log_probs[:, -1, :], dim=-1).item()
        next_word = id2word[next_word_index]
        current_tokens.append(next_word)
        if next_word == '。':
            break

In [23]:
print(''.join(current_tokens))

子曰沛视志火蒉务蒉务蒉务
